In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!apt-get install tesseract-ocr
!apt-get install libtesseract-dev
!pip install pytesseract
!pip install ultralytics
!pip install opencv-python

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 8 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (29.1 MB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 129824 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-en

In [3]:
!pip install jaxlib

In [4]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import pytesseract as pt    # Used for optical character recognition (OCR) to extract text from images
import plotly.express as px # Creating and displaying interactive plots, such as line plots, bar plots
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet # Parsing and manipulating XML data

from glob import glob   # Inding files and directories that match a specified pattern
from skimage import io  # Reading and writing images
from shutil import copy # Copying files and directories
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Model   # Creating a deep learning model
from tensorflow.keras.callbacks import TensorBoard  # Logging training information and visualizing it in TensorBoard
from tensorflow.keras.applications import InceptionResNetV2  # Loading the pre-trained InceptionResNetV2 model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input  # Defining different layers in a deep learning model
from tensorflow.keras.preprocessing.image import load_img, img_to_array  # Loading and preprocessing images
from tensorflow.keras.models import load_model  # To evaluate your model using the test set

#Import dataset

In [5]:
folderName_xml = "/content/drive/MyDrive/Colab Notebooks/project/data_xml"

In [6]:
 labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
 for name in os.listdir(folderName_xml):
    path = glob(f'{folderName_xml}/{name}')
    for filename in path:

        info = xet.parse(filename)
        root = info.getroot()
        member_object = root.find('object')
        labels_info = member_object.find('bndbox')
        xmin = int(labels_info.find('xmin').text)
        xmax = int(labels_info.find('xmax').text)
        ymin = int(labels_info.find('ymin').text)
        ymax = int(labels_info.find('ymax').text)

        labels_dict['filepath'].append(filename)
        labels_dict['xmin'].append(xmin)
        labels_dict['xmax'].append(xmax)
        labels_dict['ymin'].append(ymin)
        labels_dict['ymax'].append(ymax)

In [7]:
df = pd.DataFrame(labels_dict)
df.to_csv('labels.csv',index=False)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 433 entries, 0 to 432
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filepath  433 non-null    object
 1   xmin      433 non-null    int64 
 2   xmax      433 non-null    int64 
 3   ymin      433 non-null    int64 
 4   ymax      433 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 17.0+ KB


,filepath,xmin,xmax,ymin,ymax
0,/content/drive/MyDrive/Colab Notebooks/project...,226,419,125,173
1,/content/drive/MyDrive/Colab Notebooks/project...,134,262,128,160
2,/content/drive/MyDrive/Colab Notebooks/project...,167,240,202,220
3,/content/drive/MyDrive/Colab Notebooks/project...,152,206,147,159
4,/content/drive/MyDrive/Colab Notebooks/project...,175,214,114,131


In [8]:
folderName_img = '/content/drive/MyDrive/Colab Notebooks/project/data_img'

In [9]:
filename_xml = df['filepath'][0]
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join(folderName_img, filename_image)
    return filepath_image

In [10]:
img_path = list(df['filepath'].apply(getFilename))
img_path[:10]

['/content/drive/MyDrive/Colab Notebooks/project/data_img/Cars0.png',
 '/content/drive/MyDrive/Colab Notebooks/project/data_img/Cars1.png',
 '/content/drive/MyDrive/Colab Notebooks/project/data_img/Cars101.png',
 '/content/drive/MyDrive/Colab Notebooks/project/data_img/Cars105.png',
 '/content/drive/MyDrive/Colab Notebooks/project/data_img/Cars100.png',
 '/content/drive/MyDrive/Colab Notebooks/project/data_img/Cars107.png',
 '/content/drive/MyDrive/Colab Notebooks/project/data_img/Cars10.png',
 '/content/drive/MyDrive/Colab Notebooks/project/data_img/Cars104.png',
 '/content/drive/MyDrive/Colab Notebooks/project/data_img/Cars106.png',
 '/content/drive/MyDrive/Colab Notebooks/project/data_img/Cars102.png']

- Veritfy dataset

In [11]:
'''img_num_check = int(input())

file_path = img_path[img_num_check]
img = cv2.imread(file_path) #read the image

img = io.imread(file_path) #Read the image
fig = px.imshow(img)
fig.update_layout(width=600, height=500, margin=dict(l=10, r=10, b=10, t=10))
fig.add_shape(type='rect',
              x0=df['xmin'][img_num_check], x1=df['xmax'][img_num_check],
              y0=df['ymin'][img_num_check], y1=df['ymax'][img_num_check],
              xref='x', yref='y',line_color='cyan')'''

"img_num_check = int(input())\n\nfile_path = img_path[img_num_check]\nimg = cv2.imread(file_path) #read the image\n\nimg = io.imread(file_path) #Read the image\nfig = px.imshow(img)\nfig.update_layout(width=600, height=500, margin=dict(l=10, r=10, b=10, t=10))\nfig.add_shape(type='rect',\n              x0=df['xmin'][img_num_check], x1=df['xmax'][img_num_check],\n              y0=df['ymin'][img_num_check], y1=df['ymax'][img_num_check],\n              xref='x', yref='y',line_color='cyan')"

#Data preprocessing

In [12]:
labels = df.iloc[:,1:].values # Targeting all our values in array selecting all columns

data = []
output = []
for ind in range(len(img_path)):

    img = img_path[ind]
    img_arr = cv2.imread(img)
    h,w,d = img_arr.shape

    # Pre-procesing images
    load_image = load_img(img,target_size=(224,224)) # Resizing image to 244x244
    load_image_arr = img_to_array(load_image) # Converting image into a NumPy array with the color channels
    norm_load_image_arr = load_image_arr/255.0 # Feature scalling to each color channel have a value between 0 and 1

    # Normalization to labels
    xmin, xmax, ymin, ymax = labels[ind]
    nxmin, nxmax = xmin/w, xmax/w
    nymin, nymax = ymin/h, ymax/h
    label_norm = (nxmin, nxmax, nymin, nymax) # Normalized output

    data.append(norm_load_image_arr)
    output.append(label_norm)

#Splitting train and test set


In [13]:
X = np.array(data, dtype=np.float32)
y = np.array(output, dtype=np.float32)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size = 0.9, test_size = 0.1,
                                                    random_state = 0)

In [15]:
df.shape

(433, 5)

In [16]:
X_train.shape, y_train.shape

((389, 224, 224, 3), (389, 4))

In [17]:
X_test.shape, y_test.shape

((44, 224, 224, 3), (44, 4))

#Object Detection Model

##Initialize the InceptionResNetV2 model with pre-trained weights from ImageNet

In [18]:
inception_resnet = InceptionResNetV2(
    weights="imagenet",
    include_top=True, # Exclude the final fully connected layer
    input_tensor=Input(shape=(224,224,3)) # Set the input size as 224x224 pixels with 3 color channels
)


headmodel = inception_resnet.output # Retrieve the output from the InceptionResNetV2 model
headmodel = Flatten()(headmodel) # Add a Flatten layer to convert the output into a 1-dimensional vector
headmodel = Dense(500, activation="relu")(headmodel) # Add a fully connected layer with 500 hidden units and ReLU activation
headmodel = Dense(250, activation="relu")(headmodel) # Add another fully connected layer with 250 hidden units and ReLU activation
headmodel = Dense(4, activation='sigmoid')(headmodel)
headmodel = Dense(1)(headmodel)
# Add a final fully connected layer with 4 hidden units and sigmoid activation
# Sigmoid activation is used to predict the 4 attributes of the region of interest


# Create the complete model by specifying the inputs and outputs
model = Model(
    inputs=inception_resnet.input,
    outputs=headmodel
)

225209952/225209952 [==============================] - 6s 0us/step


In [19]:
# Compile the model
model.compile(
    loss='mse', # Use mean squared error as the loss function
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4) # Use Adam optimizer with a learning rate of 1e-4
)
model.summary() # Print a summary of the model's architecture

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

##Training Inception ResNetV2 model

In [ ]:
# Create a TensorBoard callback to visualize training progress and metrics
tfb = TensorBoard('object_detection')

training_model = model.fit(
    x=X_train, y=y_train,  # Training data
    batch_size=16,  # Number of samples per gradient update
    epochs=100,  # Number of training epochs
    validation_data=(X_test, y_test),  # Validation data
    callbacks=[tfb]  # List of callbacks to be used during training
)

Epoch 1/100
25/25 [==============================] - 136s 613ms/step - loss: 1.0644 - val_loss: 0.9361
Epoch 2/100
25/25 [==============================] - 8s 301ms/step - loss: 0.8663 - val_loss: 0.6613
Epoch 3/100
25/25 [==============================] - 8s 306ms/step - loss: 0.5867 - val_loss: 0.4009
Epoch 4/100
25/25 [==============================] - 8s 304ms/step - loss: 0.3401 - val_loss: 0.2116
Epoch 5/100
25/25 [==============================] - 8s 309ms/step - loss: 0.1709 - val_loss: 0.1130
Epoch 6/100
25/25 [==============================] - 8s 315ms/step - loss: 0.0856 - val_loss: 0.0599
Epoch 7/100
25/25 [==============================] - 8s 313ms/step - loss: 0.0560 - val_loss: 0.0464
Epoch 8/100
25/25 [==============================] - 8s 321ms/step - loss: 0.0460 - val_loss: 0.0426
Epoch 9/100
25/25 [==============================] - 8s 319ms/step - loss: 0.0428 - val_loss: 0.0441
Epoch 10/100
25/25 [==============================] - 8s 327ms/step - loss: 0.0411 - val_

In [ ]:
model.save('./License_plate_recognition.h5')

#Model evaluation

In [ ]:
model = load_model('/content/License_plate_recognition.h5')

In [ ]:
if model.evaluate(X_test, y_test) is tuple:
    loss, accuracy = model.evaluate(X_test, y_test)
else:
    loss = model.evaluate(X_test, y_test)
    accuracy = None

print('Loss:', loss)
print('Accuracy:', accuracy)

#Testing with another image

In [ ]:
model.load_weights('/content/License_plate_recognition.h5')  # Tải trọng số từ file .h5
print('Model loaded Sucessfully')

In [ ]:
path = input('Input the path of image: ')
image = load_img(path) # PIL object
image = np.array(image,dtype=np.uint8) # 8 bit array (0,255)
image1 = load_img(path,target_size=(224,224))
image_arr_224 = img_to_array(image1)/255.0  # Convert into array and get the normalized output

# Size of the orginal image
h,w,d = image.shape
print('Height of the image =',h)
print('Width of the image =',w)


In [ ]:
def object_detection(path):

    # Read image
    image = load_img(path) # PIL object
    image = np.array(image,dtype=np.uint8) # 8 bit array (0,255)
    image1 = load_img(path,target_size=(224,224))

    # Data preprocessing
    image_arr_224 = img_to_array(image1)/255.0 # Convert to array & normalized
    h,w,d = image.shape
    test_arr = image_arr_224.reshape(1,224,224,3)

    # Make predictions
    coords = model.predict(test_arr)

    # Denormalize the values
    denorm = np.array([w,w,h,h])
    coords = coords * denorm
    coords = coords.astype(np.int32)

    # Draw bounding on top the image
    xmin, xmax,ymin,ymax = coords[0]
    pt1 =(xmin,ymin)
    pt2 =(xmax,ymax)
    print(pt1, pt2)
    cv2.rectangle(image,pt1,pt2,(0,255,0),3)
    return image, coords

In [ ]:
fig = px.imshow(image)
fig.update_layout(
    width=700, height=500,
    margin=dict(l=10, r=10, b=10, t=10)
)

In [ ]:
img = np.array(load_img(path))
xmin,xmax,ymin,ymax = cods[0]
roi = img[ymin:ymax,xmin:xmax]
fig = px.imshow(roi)
fig.update_layout(width=350, height=250, margin=dict(l=10, r=10, b=10, t=10))

In [ ]:
# extract text from image
text = pt.image_to_string(roi)
print(text)